In [1]:
# todo:
# how is import redeal getting resolved? was it because it was installed?
# is there a better way to print/format than _short_str()

from redeal import *

def accept(deal):
    return deal.north.hcp >= 15

dealer = Deal.prepare({'S': 'K83 AK83 - QJT972'})

deal = dealer(accept)

deal._short_str()

'♠AT75♡Q75♢AQ86♣K6 ♠QJ92♡J964♢J7543♣ ♠K83♡AK83♢♣QJT972 ♠64♡T2♢KT92♣A8543'

In [2]:
# takes 30s
# Code from: http://datadaydreams.com/posts/a-simulation-tutorial-for-better-decisionmaking-at-bridge/
# 1. Sample random deals in a loop (the so-called "in the long run").
# 2. Accept those which satisfy constraints (as defined in the accept function).
# 3. Compute double dummy score of each competing contract.
# 4. Save result in a payoff table.

from redeal import *
from redeal import dds
from redeal.global_defs import Seat, Suit, Card, Rank, Strain

Deal.set_str_style('short')
Hand.set_str_style('short')

vuln = False
predeal = {Seat['S']: H('AJ874 A986 A A53')}
dealer = Deal.prepare(predeal)

def accept(deal):
    if deal.north.hcp > 10:
        return False
    if len(deal.north.clubs) not in [6, 7]:
        return False
    if deal.north.clubs.hcp < 3:
        return False
    return True

imps_payoff = Payoff(('pass', '3NT', '5C', '6C', '6NT'), imps)

found = 0
n = 1000
for _ in range(1000 * n):
    if found > n:
        break
    deal = dealer()
    if not accept(deal):
        continue
    found += 1
    score_3c = deal.dd_score('3CN', vuln)
    score_3n = deal.dd_score('3NS', vuln)
    score_5c = deal.dd_score('5CN', vuln)
    score_6c = deal.dd_score('6CN', vuln)
    score_6n = deal.dd_score('6NS', vuln)
    data = {
        'pass': score_3c,
        '3NT': score_3n,
        '5C': score_5c,
        '6C': score_6c,
        '6NT': score_6n,
    }
    imps_payoff.add_data(data)

imps_payoff.report()

	pass	3NT	5C	6C	6NT	
pass		-3.81	-4.82	-6.20	+4.27
		(0.17)	(0.11)	(0.28)	(0.23)
3NT	+3.81		-0.96	-4.52	+6.07
	(0.17)		(0.14)	(0.27)	(0.25)
5C	+4.82	+0.96		-4.05	+6.94
	(0.11)	(0.14)		(0.29)	(0.25)
6C	+6.20	+4.52	+4.05		+7.62
	(0.28)	(0.27)	(0.29)		(0.21)
6NT	-4.27	-6.07	-6.94	-7.62	
	(0.23)	(0.25)	(0.25)	(0.21)	


In [3]:
# 1. Sample random hands for E-W.
# 2. Count how many times the ruffing finesse line works
# 3. Count how many times the ruff-the-ten-out line works.

predeal = {
    Seat['S']: H('AJ874 A986 A A53'),
    Seat['N']: H('T6 J4 J2 KQJ9876'),
    Seat['E']: H('- KQ 4 42'),
    Seat['W']: H('- 532 83 T'),
}
dealer = Deal.prepare(predeal)

winning_line = dict(finesse=0, ruffout=0)

n = 1000
for _ in range(n):
    deal = dealer()
    if Rank['T'] in deal.west.hearts:
        winning_line['finesse'] += 1
    if len(deal.east.hearts) == 3 and Rank['T'] in deal.east.hearts:
            winning_line['ruffout'] += 1
winning_line

{'finesse': 455, 'ruffout': 274}

In [4]:
# takes 40s
# use simulation and double dummy analysis to find the best lead.

predeal = {Seat['E']: H('92 KQT K97654 42')}
dealer = Deal.prepare(predeal)
accept_north = accept

contract = Contract.from_str('6CN')

lead_payoff = Payoff(
    sorted(dds.valid_cards(dealer(), 'C', 'E'), reverse=True),
    lambda ti, tj: imps(contract.score(ti), contract.score(tj))
)

found = 0
n = 1000
for _ in range(1000*n):
    if found > n:
        continue
    deal = dealer()
    if not accept_north(deal):
        continue
    if deal.south.hcp < 16:
        continue
    has_ace = lambda hand: Rank['A'] in hand
    n_aces_west = sum(map(int, map(has_ace,
        [deal.west.spades, deal.west.hearts, deal.west.diamonds, deal.west.clubs]
    )))
    if n_aces_west > 1:
        continue
    found += 1
    lead_payoff.add_data(deal.dd_all_tricks('C', 'E'))

# display results
lead_payoff.report()

	♠9	♠2	♡K	♡T	♢K	♢9	♢7	♣4	♣2	
♠9		-0.00	-0.38	+0.29	+0.35	+0.11	+0.09	+0.18	+0.18
		(0.00)	(0.04)	(0.05)	(0.04)	(0.04)	(0.04)	(0.02)	(0.02)
♠2	+0.00		-0.38	+0.29	+0.35	+0.11	+0.09	+0.18	+0.18
	(0.00)		(0.04)	(0.05)	(0.04)	(0.04)	(0.04)	(0.02)	(0.02)
♡K	+0.38	+0.38		+0.69	+0.71	+0.48	+0.46	+0.56	+0.56
	(0.04)	(0.04)		(0.03)	(0.04)	(0.04)	(0.04)	(0.03)	(0.03)
♡T	-0.29	-0.29	-0.69		+0.04	-0.20	-0.22	-0.12	-0.12
	(0.05)	(0.05)	(0.03)		(0.05)	(0.05)	(0.05)	(0.05)	(0.05)
♢K	-0.35	-0.35	-0.71	-0.04		-0.26	-0.28	-0.18	-0.18
	(0.04)	(0.04)	(0.04)	(0.05)		(0.02)	(0.03)	(0.04)	(0.04)
♢9	-0.11	-0.11	-0.48	+0.20	+0.26		-0.02	+0.07	+0.07
	(0.04)	(0.04)	(0.04)	(0.05)	(0.02)		(0.01)	(0.03)	(0.03)
♢7	-0.09	-0.09	-0.46	+0.22	+0.28	+0.02		+0.09	+0.09
	(0.04)	(0.04)	(0.04)	(0.05)	(0.03)	(0.01)		(0.03)	(0.03)
♣4	-0.18	-0.18	-0.56	+0.12	+0.18	-0.07	-0.09		+0.00
	(0.02)	(0.02)	(0.03)	(0.05)	(0.04)	(0.03)	(0.03)		(0.00)
♣2	-0.18	-0.18	-0.56	+0.12	+0.18	-0.07	-0.09	+0.00	
	(0.02)	(0.02)	(0.03)	(0.05)	(0.04)	(0.